# コンピューティング先の操作

Azure Machine Learning SDK を使用して多くの実験を実行しましたが、それらはすべてローカル コンピューティング (この場合は、Azure Machine Learning Notebook VM) で実行されました。次に、クラウド コンピューティングを活用してコンピューティング コンテキストのスケーラビリティを向上させる方法を確認しましょう。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用データを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (前のラボで作成した場合、コードは新しいバージョンを作成します)。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                       target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                       overwrite=True, # 同じ名前の既存のファイルを置き換える
                       show_progress=True)

    # データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 表形式のデータセットを登録する
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## コンピューティング先を作成する

多くの場合、ローカル コンピューティングリソースでは、大量のデータを処理する必要がある複雑な実験や長時間実行される実験を処理するには十分でない場合があります。また、クラウドでコンピューティング リソースを動的に作成して使用する機能を活用する場合もあります。

Azure ML は、さまざまなコンピューティング先をサポートしており、このターゲットはワークペースで定義し、実験の実行に使用できます。リソースの支払いは、使用時にのみ行われます。最初の演習でワークスペースをセットアップした際に、**aml-cluster** という名前のコンピューティング クラスターを作成しました。これを確認し、作成されていない場合は作成して、トレーニングで実際に使ってみましょう。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# クラスターが存在することを確認する
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 存在しない場合は、作成する
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS1_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

## リモート コンピューティングで実験を実行する

コンピューティングを作成したら、それを使用して実験が実行できるようになります。次のコードは、実験ファイル用のフォルダーを作成します (これは前のラボから既に存在している可能性がありますが、とにかく実行してください)。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

次に、実験用 Python スクリプト ファイルを作成します。これにより、前のラボで使用したスクリプトが上書きされます。

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データを読み込む (入力データセットとして渡される)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 曲線をプロットする
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 対角 50% ラインをプロットする
plt.plot([0, 1], [0, 1], 'k--')
# モデルによって達成された FPR と TPR をプロットする
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、作成したコンピューティングで実験を実行する準備が整いました。 

> **注**: コンテナー イメージは Conda 環境で構築する必要があり、スクリプトを実行する前にクラスター ノードを起動してイメージをデプロイする必要があるため、実験にはかなり時間がかかります。糖尿病トレーニング スクリプトのような簡単な実験では、これは非効率的に見えるかもしれません。しかし、数時間かかるより複雑な実験を実行する必要があると想像してください - よりスケーラブルな計算を動的に作成すると、全体の時間が大幅に短縮される可能性があります。

In [ ]:
from azureml.core import Environment, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# 実験用 Python 環境を作成する
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # 依存関係を Azure ML に管理させる
diabetes_env.docker.enabled = True # ドッカー コンテナーを使用する

# パッケージの依存関係のセットを作成する
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                         pip_packages=['azureml-sdk','pyarrow'])

# 環境に依存関係を追加する
diabetes_env.python.conda_dependencies = diabetes_packages

# 環境を登録する (前のラボが完了していない場合)
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# Estimator を作成する
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')],
              compute_target = cluster_name, # 以前に作成したコンピューティング先を使用する
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# 実験を作成する
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()

実験の実行を待っている間に、上のウィジェットまたは [Azure Machine Learning Studio](https://ml.azure.com) でコンピューティングの状態を確認できます。次のコマンドを使用して、コンピューティングの状態を確認することもできます。

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

状態が*安定*から*サイズ変更中*に変わるまでにはしばらく時間がかかることに注意してください (コーヒーブレイクをするのによいタイミングです)。実行が完了するまでカーネルをブロックするには、下のセルを実行します。

In [ ]:
run.wait_for_completion()

実験が完了したら、実験の実行によって生成されたメトリックとファイルを取得できます。今回は、ファイルには、イメージを構築し、コンピューティングを管理するためのログが含まれます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

これで、実験によってトレーニングされたモデルを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Azure ML compute'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

>**詳細情報**: Azure Machine Learning でのコンピューティング先の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)を参照してください。